In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('C:\Shreya\HEDIS measures\Final_2018')
from datetime import datetime
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
data = pd.read_csv('ART18_final.csv', skipinitialspace = True)
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,41,42,43,44,45,46,47,48,49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,MemID,Date_S,Date_Adm,Date_Disch,DaysCov,CPT,CPTMod_1,CPTMod_2,HCPCS,CPT2,CPT2Mod,Diag_I_1,Diag_I_2,Diag_I_3,Diag_I_4,Diag_I_5,Diag_I_6,Diag_I_7,Diag_I_8,Diag_I_9,Diag_I_10,Proc_I_1,Proc_I_2,Proc_I_3,Proc_I_4,Proc_I_5,Proc_I_6,ICD_Flag,DRG,DischStatus,Rev,BillType,Nbr_times,HCFAPOS,ClaimStatus,ProvID,SuppData_x,ClaimID,StartDate,FinishDate,Dental,Drug,MHInpt,MHDN,MHAMB,CDInpt,CDDN,CDAMB,Payer,PEFlag,Ind,Gender,DOB,LName,FName,MMidName,SubID,Add1,Add2,City,State,MZip,MPhone,PFirstName,PMidName,PLastName,Race,Ethn,RaceDS,EthnDS,SpokenLang,SpokenLangSource,WrittenLang,WrittenLangSource,OtherLang,OtherSource,PDaysSup,PrServDate,NDC,Clmstat,Mquant,Dquant,SuppData_y,ProvNPI,PharmNPI,PCP,OBGYN,MHProv,EyeCProv,Dentist,Neph,Anes,NPR,PAS,ProvPres,PHAProv,Hosp,SNF,Surg,RN
0,95000,20170125,NaN,NaN,NaN,81005.0,NaN,NaN,NaN,NaN,NaN,S93.112S,A66.1,S62.231S,C83.37,S52.263Q,S46.999S,O32.0XX2,S21.102A,S72.499Q,F13.99,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,81.0,1,UNK001,N,NaN,20170211.0,20170818.0,N,Y,N,Y,N,Y,Y,Y,MMP,N,A,F,19610621,LastName,FirstName,A,95000,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,95000,20160609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G44.201,S52.042N,I70.262,B58.2,S72.466K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,20170211.0,20170818.0,N,Y,N,Y,N,Y,Y,Y,MMP,N,A,F,19610621,LastName,FirstName,A,95000,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,95000,20170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M05.229,S72.345K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,519.0,NaN,NaN,NaN,1,UNK001,N,NaN,20170211.0,20170818.0,N,Y,N,Y,N,Y,Y,Y,MMP,N,A,F,19610621,LastName,FirstName,A,95000,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,95000,20170714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S82.392K,H33.052,S27.331A,M05.069,S63.278S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,983.0,NaN,NaN,NaN,1,UNK001,N,NaN,20170211.0,20170818.0,N,Y,N,Y,N,Y,Y,Y,MMP,N,A,F,19610621,LastName,FirstName,A,95000,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95001,20171010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S62.144B,O26.711,S30.92XS,S06.369A,S52.271S,S44.00XS,S60.922S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,81.0,2,UNK001,N,NaN,19960929.0,20170312.0,N,Y,Y,Y,Y,N,Y,N,CEP,N,A,F,19870502,LastName,FirstName,A,95001,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,117.0,20191011.0,5.528902e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
score = pd.read_csv('ART18_score_modified.csv')
score.head()

,MemID,Meas,Payer,Epop,Excl,Num,Ind
0,95004,ART,MCR,1,0,1,0
1,95021,ART,MCR,1,0,0,0
2,95023,ART,MCD,1,0,0,0
3,95025,ART,MC,1,0,1,0
4,95044,ART,MCD,1,1,0,0


In [5]:
data[['MemID', 'SubID', 'Date_S']] = data[['MemID','SubID', 'Date_S']].astype('category')
score['MemID'] = score.MemID.astype('category')

In [ ]:
#def Map_unique_MemID(data, column):
    
    #data[data[column].isin(score[column])]
    
    #return data.head(10)

In [6]:
data2 = data[data.MemID.isin(score.MemID)]

In [7]:
data2.head()

,MemID,Date_S,Date_Adm,Date_Disch,DaysCov,CPT,CPTMod_1,CPTMod_2,HCPCS,CPT2,CPT2Mod,Diag_I_1,Diag_I_2,Diag_I_3,Diag_I_4,Diag_I_5,Diag_I_6,Diag_I_7,Diag_I_8,Diag_I_9,Diag_I_10,Proc_I_1,Proc_I_2,Proc_I_3,Proc_I_4,Proc_I_5,Proc_I_6,ICD_Flag,DRG,DischStatus,Rev,BillType,Nbr_times,HCFAPOS,ClaimStatus,ProvID,SuppData_x,ClaimID,StartDate,FinishDate,Dental,Drug,MHInpt,MHDN,MHAMB,CDInpt,CDDN,CDAMB,Payer,PEFlag,Ind,Gender,DOB,LName,FName,MMidName,SubID,Add1,Add2,City,State,MZip,MPhone,PFirstName,PMidName,PLastName,Race,Ethn,RaceDS,EthnDS,SpokenLang,SpokenLangSource,WrittenLang,WrittenLangSource,OtherLang,OtherSource,PDaysSup,PrServDate,NDC,Clmstat,Mquant,Dquant,SuppData_y,ProvNPI,PharmNPI,PCP,OBGYN,MHProv,EyeCProv,Dentist,Neph,Anes,NPR,PAS,ProvPres,PHAProv,Hosp,SNF,Surg,RN
35,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,T82.868A,Z77.110,S52.371N,M05.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#data2 = Map_unique_MemID(data, 'MemID')

In [8]:
data2.reset_index(drop = True, inplace = True)

In [9]:
print(data2.shape)
data2.head()

(18449, 100)


,MemID,Date_S,Date_Adm,Date_Disch,DaysCov,CPT,CPTMod_1,CPTMod_2,HCPCS,CPT2,CPT2Mod,Diag_I_1,Diag_I_2,Diag_I_3,Diag_I_4,Diag_I_5,Diag_I_6,Diag_I_7,Diag_I_8,Diag_I_9,Diag_I_10,Proc_I_1,Proc_I_2,Proc_I_3,Proc_I_4,Proc_I_5,Proc_I_6,ICD_Flag,DRG,DischStatus,Rev,BillType,Nbr_times,HCFAPOS,ClaimStatus,ProvID,SuppData_x,ClaimID,StartDate,FinishDate,Dental,Drug,MHInpt,MHDN,MHAMB,CDInpt,CDDN,CDAMB,Payer,PEFlag,Ind,Gender,DOB,LName,FName,MMidName,SubID,Add1,Add2,City,State,MZip,MPhone,PFirstName,PMidName,PLastName,Race,Ethn,RaceDS,EthnDS,SpokenLang,SpokenLangSource,WrittenLang,WrittenLangSource,OtherLang,OtherSource,PDaysSup,PrServDate,NDC,Clmstat,Mquant,Dquant,SuppData_y,ProvNPI,PharmNPI,PCP,OBGYN,MHProv,EyeCProv,Dentist,Neph,Anes,NPR,PAS,ProvPres,PHAProv,Hosp,SNF,Surg,RN
0,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,T82.868A,Z77.110,S52.371N,M05.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,19320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data2.MemID.nunique()

1562

In [11]:
def Age(data, column):  # Send DOB column
    data[column] =  pd.to_datetime(data[column]) # Converting DOB to datetime column in case if it is not
    now = pd.to_datetime('now')  # Capturing present date
    
    data['Age'] = (now.year - data[column].dt.year)
    return data.head()

In [12]:
Age(data2, 'DOB')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,MemID,Date_S,Date_Adm,Date_Disch,DaysCov,CPT,CPTMod_1,CPTMod_2,HCPCS,CPT2,CPT2Mod,Diag_I_1,Diag_I_2,Diag_I_3,Diag_I_4,Diag_I_5,Diag_I_6,Diag_I_7,Diag_I_8,Diag_I_9,Diag_I_10,Proc_I_1,Proc_I_2,Proc_I_3,Proc_I_4,Proc_I_5,Proc_I_6,ICD_Flag,DRG,DischStatus,Rev,BillType,Nbr_times,HCFAPOS,ClaimStatus,ProvID,SuppData_x,ClaimID,StartDate,FinishDate,Dental,Drug,MHInpt,MHDN,MHAMB,CDInpt,CDDN,CDAMB,Payer,PEFlag,Ind,Gender,DOB,LName,FName,MMidName,SubID,Add1,Add2,City,State,MZip,MPhone,PFirstName,PMidName,PLastName,Race,Ethn,RaceDS,EthnDS,SpokenLang,SpokenLangSource,WrittenLang,WrittenLangSource,OtherLang,OtherSource,PDaysSup,PrServDate,NDC,Clmstat,Mquant,Dquant,SuppData_y,ProvNPI,PharmNPI,PCP,OBGYN,MHProv,EyeCProv,Dentist,Neph,Anes,NPR,PAS,ProvPres,PHAProv,Hosp,SNF,Surg,RN,Age
0,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
1,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
2,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
3,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
4,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,T82.868A,Z77.110,S52.371N,M05.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50


In [14]:
data2.Age.isna().sum()

0

In [13]:
data2.Age.shape

(18449,)

In [15]:
data2.head(20)

,MemID,Date_S,Date_Adm,Date_Disch,DaysCov,CPT,CPTMod_1,CPTMod_2,HCPCS,CPT2,CPT2Mod,Diag_I_1,Diag_I_2,Diag_I_3,Diag_I_4,Diag_I_5,Diag_I_6,Diag_I_7,Diag_I_8,Diag_I_9,Diag_I_10,Proc_I_1,Proc_I_2,Proc_I_3,Proc_I_4,Proc_I_5,Proc_I_6,ICD_Flag,DRG,DischStatus,Rev,BillType,Nbr_times,HCFAPOS,ClaimStatus,ProvID,SuppData_x,ClaimID,StartDate,FinishDate,Dental,Drug,MHInpt,MHDN,MHAMB,CDInpt,CDDN,CDAMB,Payer,PEFlag,Ind,Gender,DOB,LName,FName,MMidName,SubID,Add1,Add2,City,State,MZip,MPhone,PFirstName,PMidName,PLastName,Race,Ethn,RaceDS,EthnDS,SpokenLang,SpokenLangSource,WrittenLang,WrittenLangSource,OtherLang,OtherSource,PDaysSup,PrServDate,NDC,Clmstat,Mquant,Dquant,SuppData_y,ProvNPI,PharmNPI,PCP,OBGYN,MHProv,EyeCProv,Dentist,Neph,Anes,NPR,PAS,ProvPres,PHAProv,Hosp,SNF,Surg,RN,Age
0,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
1,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
2,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
3,95004,20170209,20170209.0,20170214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S92.356P,T65.1X3D,S72.441B,S37.99XA,S82.192D,M06.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,202.0,065K,NaN,31.0,1,SNF001,N,NaN,20170222.0,20181231.0,N,Y,Y,Y,Y,N,N,Y,SN2,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
4,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,T82.868A,Z77.110,S52.371N,M05.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,70.0,20170930.0,6.050530e+10,1.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
5,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,T82.868A,Z77.110,S52.371N,M05.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,1,UNK001,N,NaN,19350613.0,20170221.0,N,Y,Y,Y,Y,N,N,Y,MC,N,A,F,1970-01-01 00:00:00.019320204,LastName,FirstName,A,95004,Address1 ...,Address2 ...,City,AB,99999,9999999999,PFirstName,P,PLastName,1,12,21,91,31,41,51,61,71,81,59.0,20170324.0,5.910796e+08,2.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
6,95004,20170427,NaN,NaN,NaN,NaN,NaN,NaN,G0438,NaN,NaN,N97.8,I80.222,S52.335M,M84.369G,

In [16]:
data2.shape

(18449, 101)

In [17]:
final_2018 = pd.DataFrame(index=np.arange(len(data2.index)), columns=['MemID'])

In [18]:
final_2018['MemID'] = data2['MemID']

In [19]:
final_2018.shape

(18449, 1)

In [20]:
final_2018['Proc_Count'] = data2[['HCPCS', 'CPT', 'CPTMod_1', 'CPTMod_2', 'CPT2', 'CPT2Mod', 'Proc_I_1', 'Proc_I_2', 'Proc_I_3', 'Proc_I_4', 'Proc_I_5', 'Proc_I_6']].nunique(axis = 1, dropna = True)
final_2018['Diag_Count'] = data2[['Diag_I_1', 'Diag_I_2', 'Diag_I_3', 'Diag_I_4', 'Diag_I_5', 'Diag_I_6', 'Diag_I_7', 'Diag_I_8', 'Diag_I_9', 'Diag_I_10']].nunique(axis = 1, dropna = True)

In [26]:
final_2018.head(10)

,MemID,Proc_Count,Diag_Count
0,95004,0,6
1,95004,0,6
2,95004,0,6
3,95004,0,6
4,95004,1,8
5,95004,1,8
6,95004,1,8
7,95004,1,8
8,95021,0,1
9,95021,0,1


In [27]:
final_2018.groupby('MemID').sum()

,Proc_Count,Diag_Count
MemID,,
95000,0,0
95001,0,0
95002,0,0
95003,0,0
95004,4,56
...,...,...
107733,0,0
107734,0,0
107735,0,0
